In [ ]:
#default_exp spot

# SPOT

> Working through conversion to cogs, upload to object storage, stac item metadata parsing, etc. for SPOT 6/7 datasets

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
from glob import glob
import time
import numpy as np
from datetime import datetime
import json

import xmltodict
import pystac
from pystac import STAC_IO
from pystac.extensions.eo import Band
import geopandas as gpd

from sac_stac.utils import sedas_client, sedas_find_datasets, sedas_download, sedas_extract
from sac_stac.utils import cogmosaicbands
from sac_stac.utils import s3_upload_dir, s3_list_objects_paths, clean_up
from sac_stac.utils import pystac_setIO, create_uri

In [ ]:
import pandas as pd

## **Preparation**: ***Download, cloud-optimise, upload***

### **Prep Function**

Download and use basic gdal tools to mosaic any tiles into single images, convert to cog and upload to our object storage.

In [ ]:
#export
def prep_spot(sedas_supplierId, inter_dir="/tmp/data/intermediate/", 
                  s3_bucket="public-eo-data", s3_dir="uksa-ssgp/spot/"):
    try:
        inter_dir = f"{inter_dir}{sedas_supplierId}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        scene_name = sedas_supplierId
        down_zip = f"{inter_dir}{scene_name}.zip"
        scene_dir = f"{down_zip[:-4]}/"
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {scene_name} within {inter_dir}")
        # find & download
        sedas_scene_res = sedas_client().search_product(sedas_supplierId)[0]
        sedas_download([sedas_scene_res], inter_dir)
        sedas_extract(down_zip, scene_dir)
        # sensor-specific band mosaicing and cogifying
        imgs_ms = glob(f"{scene_dir}*/*/*MS_001*/*.TIF")
        imgs_pan = glob(f"{scene_dir}*/*/*P_001*/*.TIF")
        cogmosaicbands(imgs_pan, 1, imgs_pan[0][:-20])
        cogmosaicbands(imgs_ms, 4, imgs_ms[0][:-20])
        # upload
        s3_upload_dir(scene_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {scene_name} at {s3_dir}{scene_name}/")
        clean_up(inter_dir)
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
        clean_up(inter_dir)

### **Iteration with examples**

Some samples used to iterate creation of the prep function.

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "SPOT"
                            )
pd.DataFrame(result['products']).head(2)

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,ff3aa1470eb5ed57d7c5a6e05db400b7,UKSA_SPOT161_SO18034609-61-01_DS_SPOT6_2018101...,ARCHIVE,SPOT-6,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-2.820905 50.931699,-1.870556 50.9345...",2018-10-19T10:39:14Z,...,4.313839e+09,0.0,0.0,6.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '63...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,c0982f29580eb726f04c9af6bdebb376,UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_2018101...,ARCHIVE,SPOT-6,NAOMI-MS/PAN,0.000,Optical,8.0,"POLYGON((-1.918729 51.158248,-0.864598 51.1527...",2018-10-10T10:58:21Z,...,3.463299e+09,25.0,13.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '73...",https://sedasdm.satapps.org/datamgr/datamgr.ph...


In [ ]:
#hide

# done = []
# for p in pd.DataFrame(result['products']).supplierId.values:
# #     if not True in [i.split('_')[1] in p for i in done]:
#     prep_spot(p)

Example of testing the first one.

In [ ]:
# prep_spot("UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140")

### **Full Job Lists**

Once happy with `prep_spot` we create a list of jobs from search results of `sedas_find_datasets` to submit to a redis queue as per `rediswq`.

We can test these via redis within `spot_prep_worker`.

Sometimes we run as campaigns via K8s.

In [ ]:
jobs_dir = "/tmp/data/"
td = datetime.today()
td = td.strftime('%Y')+td.strftime('%m')+td.strftime('%d')
td

'20201111'

In [ ]:
with open(f"{jobs_dir}/JOBLIST_{td}_spot.txt", 'w') as t:
    for v in pd.DataFrame(result['products']).supplierId.values:
        t.write("rpush jobPL"+" '{"+'"sedas_supplierId": "'+ v +'"'"}'" + '\n')

## **STAC metadata**: ***core & extensions***

With the cogs above hosted object storage we can go create some tools for building a STAC *Collection* from them, to be used within in **insert nb** to build a *Catalog* of different Catapult *Collections*. As per the nb these initially sit within a *static* STAC alongside the actual datasets on the object storage. However we plan on hosting via a STAC compliant API - probs [pygeoapi](https://pygeoapi.io/).

In [ ]:
obj_paths_list = s3_list_objects_paths('public-eo-data', 'uksa-ssgp/spot/')

In [ ]:
# get unique Item / scene names (third dir from path)
scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
scene_names[:5]

['UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140',
 'UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140',
 'UKSA_SPOT161_SO18034609-61-01_DS_SPOT6_201810191039150_FR1_FR1_SV1_SV1_W002N51_01709',
 'UKSA_SPOT287_SO18034610-86-01_DS_SPOT7_201809241031594_FR1_FR1_FR1_FR1_W002N52_01222',
 'UKSA_SPOT288_SO18034610-87-01_DS_SPOT7_201809271058114_FR1_FR1_SV1_SV1_W001N52_02845']

### **Individual example for iteration**

can just work with one

In [ ]:
scene_name = scene_names[0]
scene_name

'UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140'

and objects associated with that scene

In [ ]:
scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
scene_obj_paths[:2]

['uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/DELIVERY.PDF',
 'uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/INDEX.HTM']

set our own I/O for pystac

In [ ]:
#export
pystac_setIO()

### **Functions for** ***Item*** **metadata**

In [ ]:
#export
def spot_get_dt(scene_name):
    return datetime.strptime(scene_name.split('_')[5][:14], '%Y%m%d%H%M%S')

In [ ]:
spot_get_dt(scene_name)

datetime.datetime(2018, 10, 10, 10, 58, 9)

In [ ]:
#export
def spot_parsemeta(scene_name, scene_obj_paths):
    meta_path = [i for i in scene_obj_paths if (i.endswith('.XML')) & (os.path.basename(i).startswith('DIM'))][0]
    return xmltodict.parse(pystac.STAC_IO.read_text(create_uri(meta_path)))

In [ ]:
meta = spot_parsemeta(scene_name, scene_obj_paths)
meta['Dimap_Document'].keys()

odict_keys(['@xmlns:xlink', '@xmlns:xsi', '@xsi:noNamespaceSchemaLocation', '@version', 'Metadata_Identification', 'Dataset_Identification', 'Dataset_Content', 'Product_Information', 'Coordinate_Reference_System', 'Geoposition', 'Processing_Information', 'Raster_Data', 'Radiometric_Data', 'Geometric_Data', 'Quality_Assessment', 'Dataset_Sources'])

In [ ]:
#export
def spot_get_crs(metadata):
    return int(metadata['Dimap_Document']['Coordinate_Reference_System']['Projected_CRS']['PROJECTED_CRS_NAME'][:5])

In [ ]:
crs = spot_get_crs(meta)
crs

27700

In [ ]:
#export
def spot_get_geom(scene_paths, native_epsg):
    nat_crs = {"init": f"epsg:{native_epsg}"}
    roi_path = [i for i in scene_paths if (i.endswith('1_MSK.GML') * os.path.basename(i).startswith('ROI'))][0]
    roi_uri = create_uri(roi_path)
    g = gpd.read_file(roi_uri)
    g.crs = f"EPSG:{native_epsg}"
    return json.loads(g.to_crs('EPSG:4326').to_json(show_bbox=True))['features'][0]['geometry']

In [ ]:
spot_get_geom(scene_obj_paths, crs)

{'type': 'Polygon',
 'coordinates': [[[-0.6499519273083667, 52.08844089007527],
   [-0.6448428681559101, 52.088382273153236],
   [-0.6446263981696051, 51.757267668413746],
   [-0.644682404135545, 51.755396785700185],
   [-0.6450817810991609, 51.75512292935542],
   [-1.6964988848377145, 51.685308252199924],
   [-1.6970101211854633, 51.685309570681724],
   [-1.6975136320308248, 51.685588611054996],
   [-1.6977238663030947, 51.85203916138853],
   [-1.6978394095956657, 51.94347348226774],
   [-1.6973325231656649, 52.018813144847385],
   [-0.6499519273083667, 52.08844089007527]]]}

In [ ]:
#export
def spot_get_bbox(metadata):
    lons = [float(i['LON']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    lats = [float(i['LAT']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    return [min(lons), min(lats), max(lons), max(lats)]

In [ ]:
spot_get_bbox(meta)

[-1.69960277718, 51.6779380171, -0.634664143202, 52.095806321]

In [ ]:
#export
def spot_get_gsd(metadata):
    across = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ACROSS_TRACK'])
    along = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ALONG_TRACK'])
    return round(( across + along ) / 2, 2)

In [ ]:
spot_get_gsd(meta)

9.93

In [ ]:
#export
def spot_get_cloudcover(metadata):
    return round(float(metadata['Dimap_Document']['Dataset_Content']['CLOUD_COVERAGE']['#text']),2)

In [ ]:
spot_get_cloudcover(meta)

0.0

**Can now create** ***Item*** **containing core STAC metadata.**

In [ ]:
#export
def spot_create_item(scene_name, scene_obj_paths):

    meta = spot_parsemeta(scene_name, scene_obj_paths)
    
    crs = spot_get_crs(meta)
    
    item = pystac.Item(id=scene_name,
                      datetime=spot_get_dt(scene_name),
                      geometry=spot_get_geom(scene_obj_paths, crs),
                      bbox=spot_get_bbox(meta),
                      properties={})

    item.common_metadata.gsd = spot_get_gsd(meta)

    item.ext.enable('eo')
    item.ext.eo.cloud_cover = spot_get_cloudcover(meta)

    item.ext.enable('projection')
    item.ext.projection.epsg = spot_get_crs(meta)

    return item

In [ ]:
example_item = spot_create_item(scene_name, scene_obj_paths)
example_item.validate()

In [ ]:
example_item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140',
 'properties': {'gsd': 9.93,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700,
  'datetime': '2018-10-10T10:58:09Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-0.6499519273083667, 52.08844089007527],
    [-0.6448428681559101, 52.088382273153236],
    [-0.6446263981696051, 51.757267668413746],
    [-0.644682404135545, 51.755396785700185],
    [-0.6450817810991609, 51.75512292935542],
    [-1.6964988848377145, 51.685308252199924],
    [-1.6970101211854633, 51.685309570681724],
    [-1.6975136320308248, 51.685588611054996],
    [-1.6977238663030947, 51.85203916138853],
    [-1.6978394095956657, 51.94347348226774],
    [-1.6973325231656649, 52.018813144847385],
    [-0.6499519273083667, 52.08844089007527]]]},
 'links': [],
 'assets': {},
 'bbox': [-1.69960277718, 51.6779380171, -0.634664143202, 52.095806321],
 'stac_extensions': ['eo',

### **Functions for** ***Asset*** **metadata**

Once we have *Item* level metadata we can add the actual *Assets*. There are a few constants used within these functions at the *Item* level (i.e. band info) and criteria for finding within the object paths.

In [ ]:
#export
spot_bands = [Band.create(name='Panchromatic', description='Panchromatic: 450 - 745 nm', common_name='pan'),
              Band.create(name='Blue', description='Blue: 450 - 520 nm', common_name='blue'),
              Band.create(name='Green', description='Green: 530 - 590 nm', common_name='green'),
              Band.create(name='Red', description='Red: 625 - 695 nm', common_name='red'),
              Band.create(name='Near-Infrared', description='Near-Infrared: 760 - 890 nm', common_name='nir')]

In [ ]:
#export
spot_band_refs = {
    'Panchromatic':{'ends':'_band1', 'dif':'_P_', 'id':'B0'},
    'Blue':{'ends':'_band1', 'dif':'_MS_', 'id':'B1'},
    'Green':{'ends':'_band2', 'dif':'_MS_', 'id':'B2'},
    'Red':{'ends':'_band3', 'dif':'_MS_', 'id':'B3'},
    'Near-Infrared':{'ends':'_band4', 'dif':'_MS_', 'id':'B4'}
}

In [ ]:
spot_band_refs

{'Panchromatic': {'ends': '_band1', 'dif': '_P_', 'id': 'B0'},
 'Blue': {'ends': '_band1', 'dif': '_MS_', 'id': 'B1'},
 'Green': {'ends': '_band2', 'dif': '_MS_', 'id': 'B2'},
 'Red': {'ends': '_band3', 'dif': '_MS_', 'id': 'B3'},
 'Near-Infrared': {'ends': '_band4', 'dif': '_MS_', 'id': 'B4'}}

Need to find the actual asset path for a given band. (Note that we typically store COGs per-band.)

In [ ]:
#export
def spot_find_band_path(band_name, scene_obj_paths):
    matched_paths = [ o for o in scene_obj_paths if (o.endswith(f"{spot_band_refs[band_name]['ends']}.tif")) & (f"{spot_band_refs[band_name]['dif']}" in o) ]
    if len(matched_paths) > 1:
        raise Exception(f"Found too many matches: {matched_paths}")
#     elif len(matched_paths) == 0: # should probably add something for when no asset is found...
#         raise Warning(f"")
    return matched_paths[0]

In [ ]:
spot_find_band_path('Blue', scene_obj_paths)

'uksa-ssgp/spot/UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140/PROD_SPOT6_001/VOL_SPOT6_001_A/IMG_SPOT6_MS_001_A/IMG_SPOT6_MS_201810101058095_ORT_band1.tif'

Then we can add each asset expected to be found and test with the example item taken from above.

In [ ]:
#export
def spot_add_assets2item(item, scene_obj_paths):
    for band in spot_bands:
#         print(band.name)

        band_path = spot_find_band_path(band.name, scene_obj_paths)
        band_url = create_uri(band_path)
#         print(band_url)

        asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
        item.ext.eo.set_bands([band], asset)
        item.add_asset(spot_band_refs[band.name]['id'], asset)    
        
    return item

In [ ]:
example_item_with_assets = spot_add_assets2item(example_item, scene_obj_paths)

Now we can see the completed *Item* stac record.

In [ ]:
example_item_with_assets.validate()

In [ ]:
example_item_with_assets.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140',
 'properties': {'gsd': 9.93,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700,
  'datetime': '2018-10-10T10:58:09Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-0.6499519273083667, 52.08844089007527],
    [-0.6448428681559101, 52.088382273153236],
    [-0.6446263981696051, 51.757267668413746],
    [-0.644682404135545, 51.755396785700185],
    [-0.6450817810991609, 51.75512292935542],
    [-1.6964988848377145, 51.685308252199924],
    [-1.6970101211854633, 51.685309570681724],
    [-1.6975136320308248, 51.685588611054996],
    [-1.6977238663030947, 51.85203916138853],
    [-1.6978394095956657, 51.94347348226774],
    [-1.6973325231656649, 52.018813144847385],
    [-0.6499519273083667, 52.08844089007527]]]},
 'links': [],
 'assets': {'B0': {'href': 'http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/spot/UKSA_SPOT155_SO18034609

### **Compiling** ***Items*** **into a** ***Collection*** 

We naurally want to apply the above tools to all SPOT-related *Items* and their *Assets* in order to build a *Collection* that can sit within another *Collection* i.e. comprised of other UKSA/SSGP-procured datasets and maybe an overall Satellite Applications Catapult *Catalog* covering all of our internal and external-facing geospatial datasets.

In [ ]:
#export
def spot_create_collection(spot_dir, bucket='public-eo-data'):
    
    collection_id = 'uksa-ssgp-spot'
    collection_title = 'SSGP-procured Spot 6 & 7 images over the UK'
    collection_description = '''### UKSA / SSGP SPOT 6 & 7

    A collection of SPOT 6 & 7 images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).
    '''
    
    # initially arbitrary as updated later
    spatial_extent = pystac.SpatialExtent([[-7.57216793459, 49.959999905, 1.68153079591, 58.6350001085]])
    temporal_extent = pystac.TemporalExtent([[datetime(2011, 12, 16), None]])
    collection_extent = pystac.Extent(spatial_extent, temporal_extent)
    
    collection = pystac.Collection(id=collection_id,
                                   title=collection_title,
                                   description=collection_description,
                                   extent=collection_extent)
    
    collection.providers = [
        pystac.Provider(name='Airbus Defence & Space', roles=['producer'], url='https://www.airbus.com/space.html'),
        pystac.Provider(name='UK Space Agency', roles=['licensor'], url='https://www.gov.uk/government/organisations/uk-space-agency'),
        pystac.Provider(name='Satellite Applications Catapult', roles=['processor'], url='https://sa.catapult.org.uk/'),
        pystac.Provider(name='Satellite Applications Catapult', roles=['host'], url='https://sa.catapult.org.uk/')
    ]
    
    obj_paths_list = s3_list_objects_paths(bucket, spot_dir)
    scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
    
    for scene_name in scene_names:
        
        scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
        
        item = spot_create_item(scene_name, scene_obj_paths)
        item = spot_add_assets2item(item, scene_obj_paths)
        
        collection.add_item(item)
    
    collection.update_extent_from_items()
        
    return collection
    

In [ ]:
example_collection = spot_create_collection('uksa-ssgp/spot/')

In [ ]:
example_collection.describe()

* <Collection id=uksa-ssgp-spot>
  * <Item id=UKSA_SPOT155_SO18034609-55-01_DS_SPOT6_201810101058095_FR1_FR1_FR1_FR1_W001N52_01140>
  * <Item id=UKSA_SPOT156_SO18034609-56-01_DS_SPOT6_201810101058206_FR1_FR1_FR1_FR1_W001N51_01140>
  * <Item id=UKSA_SPOT161_SO18034609-61-01_DS_SPOT6_201810191039150_FR1_FR1_SV1_SV1_W002N51_01709>
  * <Item id=UKSA_SPOT287_SO18034610-86-01_DS_SPOT7_201809241031594_FR1_FR1_FR1_FR1_W002N52_01222>
  * <Item id=UKSA_SPOT288_SO18034610-87-01_DS_SPOT7_201809271058114_FR1_FR1_SV1_SV1_W001N52_02845>
  * <Item id=UKSA_SPOT289_SO18034610-88-01_DS_SPOT7_201809271058350_FR1_FR1_SV1_SV1_W002N52_03251>
  * <Item id=UKSA_SPOT290_SO18034610-89-01_DS_SPOT7_201809271059006_FR1_FR1_FR1_FR1_W002N52_03332>
  * <Item id=UKSA_SPOT291_SO18034610-90-01_DS_SPOT7_201809271059251_FR1_FR1_SV1_SV1_W002N51_04469>


In [ ]:
example_collection.to_dict()

{'id': 'uksa-ssgp-spot',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP SPOT 6 & 7\n\n    A collection of SPOT 6 & 7 images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n    ',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Spot 6 & 7 images over the UK',
 'extent': {'spatial': {'bbox': [[-2.83129686995,
     50.4427331374,
     -0.240355992445,
     52.473109175]]},
  'temporal': {'interval': [['20

Note: We normalised hrefs and save in a different notebook.

## Export

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_rediswq.ipynb.
Converted 00_utils.ipynb.
Converted 01A_pleiades.ipynb.
Converted 01B_pleiades_prep_worker.ipynb.
Converted 02A_spot.ipynb.
Converted index.ipynb.
